# #30DayMapChallenge - Day 1: Points

Use Ordnance Survey's Downloads API to find and download their CodePoint Open product as a zipped GeoPackage.

In [1]:
from requests import get
from pathlib import Path

# Get a listing of all of Ordnance Survey's products
os_downloads_api_url = 'https://api.os.uk/downloads/v1/products'
osDownloadsEntries = get(os_downloads_api_url).json()

# Filter that list to just the CodePoint Open product and find the download
# URLs for it.
codePointEntry = next(filter(lambda entry: entry['id'] == 'CodePointOpen', osDownloadsEntries))
codePointFullEntry = get(codePointEntry['url']).json()
codePointDownloads = get(codePointFullEntry['downloadsUrl']).json()

# Find the GeoPackage version of CodePoint Open.
codePointGeoPackageEntry = next(filter(lambda entry: entry['format'] == 'GeoPackage', codePointDownloads))

# Create a directory (if it doesn't already exist) to store the GeoPackage.
p = Path("./data/os-codepointopen")
p.mkdir(parents=True, exist_ok=True)

dl_file = p / codePointGeoPackageEntry['fileName']
target_file = dl_file.with_suffix('.gpkg')
json_file = dl_file.with_suffix('.geojson')

# Check to see if we've previously downloaded the CodePoint Open GeoPackage.
if dl_file.exists() == False and target_file.exists() == False and json_file.exists() == False:
    
    # If we haven't, download the file...
    with get(codePointGeoPackageEntry['url'], stream=True) as r:
        r.raise_for_status()
        # ...and save it.
        with open(dl_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)

Extract the GeoPackage from inside the zip file and save it to disk.

In [2]:
from zipfile import ZipFile

# Check if we've already got an unzipped copy of the GeoPackage.
if target_file.exists() == False and json_file.exists() == False:
    # If not, unzip it to the target destination.
    with ZipFile(dl_file, 'r') as zip:
        geopackage = zip.getinfo('data/codepo_gb.gpkg')
        geopackage.filename = str(target_file)
        zip.extract(geopackage)

Load the geopackage, then since we only want postcodes in the IV2 Outward Code, filter the incoming geodataframe to only include those rows. Ordnance Survey data arrives in OSGB eastings and northings, but we want it in WGS84 GPS style coordinates so we need to convert from EPSG:27700 to EPSG:4326. Save it as a geojson for later ease of use.

In [3]:
import geopandas as gpd

# Check if we've already trimmed the file down.
if json_file.exists() == False:
    
    # If not, read the GeoPackage.
    codepoint_gdf = gpd.read_file(target_file)
    
    # Slice just the two columns we're interested in.
    sub_gdf = codepoint_gdf[['Postcode','geometry']]

    # Get just the target rows, convert them to EPSG:4326 and save them.
    sub_gdf[
        sub_gdf.Postcode.str.startswith('IV2 ')
    ].to_crs(epsg=4326).to_file(json_file, driver='GeoJSON')

In [4]:
# Read the trimmed down file.
iv2_gdf = gpd.read_file(json_file)
iv2_gdf.sample(3)

,Postcode,geometry
206,IV2 3SJ,POINT (-4.19337 57.47077)
725,IV2 5NA,POINT (-4.18159 57.47738)
699,IV2 5HH,POINT (-4.18799 57.46352)


Render a map with the 'IV2 3B?' postcodes as green markers and all other 'IV2' postcodes as red markers.

In [5]:
import folium
from folium.vector_layers import CircleMarker

def add_marker(folium_map, postcode, position, line_c, fill_c):
    """Helper function for adding markers to a map."""
    folium_map.add_child(CircleMarker(
        location=[position.y, position.x],
        popup="""
                <span style="
                    font-weight:700;
                    white-space: nowrap;
                ">
            """ + postcode + '</span>',
        color=line_c,
        fill=True,
        fill_opacity=1.0,
        fill_color=fill_c
    ))

# Grab all the IV2 3B? entries
iv2_3b_gdf = iv2_gdf[iv2_gdf.Postcode.str.startswith('IV2 3B')]

# Grab all the other IV2 entries
iv2_not3b_gdf = iv2_gdf[~iv2_gdf.Postcode.str.startswith('IV2 3B')]

# Create a folium map centred on the middle of the IV2 3B? entries.
map = folium.Map(
    location=[57.47,-4.2],
    zoom_start=14,
    tiles='Stamen Toner',
    control_scale=True)

# Plot the NOT IV2 3B? entries in red.
for entry in iv2_not3b_gdf.index:
    add_marker(
        map,
        iv2_not3b_gdf.Postcode[entry], iv2_not3b_gdf.geometry[entry],
        'red', 'lightcoral'
    )

# Plot the IV2 3B? entries in green.
for entry in iv2_3b_gdf.index:
    add_marker(
        map,
        iv2_3b_gdf.Postcode[entry], iv2_3b_gdf.geometry[entry],
        'green', 'lightgreen'
    )

map